In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.1-8b-instant"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://stripe.com/jobs/listing/software-engineering-new-grad/7176975/apply")
page_data = loader.load().pop().page_content

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)
print(res)

content='```python\nimport re\nimport json\n\n# Scraped text\ntext = """\nSoftware Engineering, New Grad\nJobs\nOur opportunity\nLife at Stripe\nBenefits\nUniversity\nSee open roles\nOpen mobile navigation\nJobs\nOur opportunity\nLife at Stripe\nBenefits\nUniversity\nSee open roles\nClose mobile navigation\nRoles at Stripe\nRole details\nApplication\nSoftware Engineering, New Grad\n"""\n\n# Regular expression pattern to extract job postings\npattern = r"Software Engineering, New Grad(?:(?:\\n.*\\n)*?(?:Our opportunity|Life at Stripe|Benefits|University|See open roles|Close mobile navigation|Roles at Stripe|Role details|Application))"\n\n# Find all job postings\njob_postings = re.findall(pattern, text, re.DOTALL)\n\n# Initialize an empty list to store the extracted job postings\njobs = []\n\n# Loop through each job posting\nfor posting in job_postings:\n    # Split the posting into lines\n    lines = posting.split(\'\\n\')\n    \n    # Initialize an empty dictionary to store the extract